# Data Collection

## Set up logging

In [1]:
import logging
from ipylogging import DisplayHandler, HTMLFormatter

handler = DisplayHandler()
handler.setFormatter(HTMLFormatter())

log = logging.getLogger()
log.addHandler(handler)
log.setLevel(logging.INFO)

## Global variables

In [2]:
from pathlib import Path

data_dir = Path('../data')
cache_dir_github = data_dir.joinpath('github')
bots_dataset_path = data_dir.joinpath('bots-dataset.csv')
bots_issues_dir = data_dir.joinpath('bots-issues')

#github_token = open('../gh-token.txt','r').readlines()[0].strip()


## Step 1 - Collect bots from [Golzadeh et al.](https://zenodo.org/record/4000388)'s dataset

Download dataset from [Golzadeh et al.](https://zenodo.org/record/4000388)

In [3]:
import urllib
import gzip
import os

url_bot_dataset = "https://zenodo.org/record/4000388/files/groundtruthbots.csv.gz"
path_bot_dataset = data_dir.joinpath('groundtruthbots.csv')

gz_path, _ = urllib.request.urlretrieve(url_bot_dataset)
with gzip.open(gz_path, "rb") as f_in, open(path_bot_dataset, "wb") as f_out:
    f_out.write(f_in.read())

Extract bots

In [ ]:
from minirig import load_csv_dataset, save_csv_dataset

bot_dataset = load_csv_dataset(path_bot_dataset)
bot_dataset = [{'account': row['account']} for row in bot_dataset if row['type'] == 'Bot']
save_csv_dataset(bots_dataset_path, data=bot_dataset)

## Step 2 - Collect the number of issues per bot

In [ ]:
from minirig import load_csv_dataset, save_csv_dataset
from minirig import GHRequests

bot_dataset = load_csv_dataset(bots_dataset_path)
gh_api = GHRequests(token=github_token,cache_dir=cache_dir_github)

for bot in bot_dataset:
    try:
        bot['issue_count'] = gh_api.get_number_issues_involving_user(bot['account'], force=True)
    except:
        bot['issue_count'] = 'na'

bot_dataset.sort(reverse=True, key=lambda x: -1 if x['issue_count'] == 'na' else x['issue_count'])
save_csv_dataset('../data/new-bot-dataset.csv', data=bot_dataset, s)

## Step 3 - Download issues for each bot

In [ ]:
from IPython.display import clear_output
from minirig import GHRequests, load_csv_dataset
import pandas as pd
import json

bot_dataset = load_csv_dataset(bots_dataset_path)
gh_api = GHRequests(token=github_token,cache_dir=cache_dir_github)

issues_errors = []

for bot in bot_dataset:
    cnt = 1
    for issue in gh_api.get_issues_involving_user(bot['account']):
        
        bot_issue_path = bots_issues_dir.joinpath(bot['account']).joinpath(issue['html_url'].replace('https://github.com/',''))
        owner, project = issue['repository_url'].replace('https://api.github.com/repos/', '').split('/')
        
        if not bot_issue_path.joinpath('json').exists():    
            full_issue = gh_api.get_issue_info(issue['number'], owner, project)  
            bot_issue_path.mkdir(parents=True, exist_ok=True)
            with open(bot_issue_path.joinpath('json'), 'w') as f:
                json.dump(full_issue, f)

        clear_output()
        logging.info(f"{bot['account']}: {cnt} of {bot['issue_count']}")
        cnt += 1

In [ ]:
from IPython.display import clear_output
from minirig import GHRequests, load_csv_dataset
import pandas as pd
import json

bot_dataset = load_csv_dataset(bots_dataset_path)
gh_api = GHRequests(token=github_token,cache_dir=cache_dir_github)

vaadin_bot = [x for x in bot_dataset if x['account'] == 'vaadin-bot']

for bot in vaadin_bot:
    
    for issue in gh_api.get_issues_involving_user(bot['account'], force= True):
        
        bot_issue_path = bots_issues_dir.joinpath(bot['account']).joinpath(issue['html_url'].replace('https://github.com/',''))
        owner, project = issue['repository_url'].replace('https://api.github.com/repos/', '').split('/')
        
        if not bot_issue_path.joinpath('json').exists():    
            full_issue = gh_api.get_issue_info(issue['number'], owner, project)  
            bot_issue_path.mkdir(parents=True, exist_ok=True)
            with open(bot_issue_path.joinpath('json'), 'w') as f:
                json.dump(full_issue, f)

        clear_output()
        logging.info(f"{bot['account']}: {cnt} of {bot['issue_count']}")
        cnt += 1

## Step 4 - Download comments for each issue

In [ ]:
from IPython.display import clear_output
from minirig import GHRequests, load_csv_dataset
import pandas as pd
import json

bot_dataset = load_csv_dataset(bots_dataset_path)
gh_api = GHRequests(token=github_token,cache_dir=cache_dir_github)

issues_errors = []

for bot in bot_dataset:
    cnt = 1
    for issue in gh_api.get_issues_involving_user(bot['account']):
        bot_issue_path = bots_issues_dir.joinpath(bot['account']).joinpath(issue['html_url'].replace('https://github.com/','')).joinpath('comments')
        bot_issue_path.mkdir(parents=True, exist_ok=True)
        owner, project = issue['repository_url'].replace('https://api.github.com/repos/', '').split('/')
        comments = gh_api.get_comments_per_issue(issue['number'], owner, project, force = True)           
        with open(bot_issue_path.joinpath('json'), 'w') as f:
            json.dump(comments, f)
        clear_output()
        logging.info(f"{bot['account']}: {cnt} of {bot['issue_count']}")
        print(bot_issue_path)
        cnt += 1

## Step 6 - Labeling sections with [Li et al.](http://doi.org/10.1007/s10664-022-10128-3)'s Model 

In [3]:
from model_li2022_emse import *
nltk.download('punkt')
model1_li2022_emse = Model1_IssueTracker_Li2022_ESEM('../model/mode1-issue-tracker-li2022-emse/model1-issue-tracker-li2022-esem-weight_file.hdf5', 
                 '../model/mode1-issue-tracker-li2022-emse/model1-issue-tracker-li2022-esem-word_embedding_file.bin')

2023-05-15 16:16:16.184872: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package punkt to /home/mambauser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Loading model ../model/mode1-issue-tracker-li2022-emse/model1-issue-tracker-li2022-esem-weight_file.hdf5...


In [4]:
def load_issues_per_bot(bot, issue_count):
    issues = []
    cnt = 1
    for owner in os.listdir(bots_issues_dir.joinpath(bot)):
        for project in os.listdir(bots_issues_dir.joinpath(bot).joinpath(owner)):
            for issue in os.listdir(bots_issues_dir.joinpath(bot).joinpath(owner).joinpath(project).joinpath('issues')):
                issue_path = bots_issues_dir.joinpath(bot).joinpath(owner).joinpath(project).joinpath('issues').joinpath(issue)
                try:
                    with open(issue_path.joinpath('json')) as f:
                        issue_file = json.load(f)
                        yield {'path':issue_path, 'content':issue_file}
                except:
                    continue

def load_issues_comments_per_bot(bot, issue_count):
    issues = []
    cnt = 1
    for owner in os.listdir(bots_issues_dir.joinpath(bot)):
        for project in os.listdir(bots_issues_dir.joinpath(bot).joinpath(owner)):
            for issue in os.listdir(bots_issues_dir.joinpath(bot).joinpath(owner).joinpath(project).joinpath('issues')):
                comments_path = bots_issues_dir.joinpath(bot).joinpath(owner).joinpath(project).joinpath('issues').joinpath(issue).joinpath('comments')
                try:
                    with open(comments_path.joinpath('json')) as f:
                        comments_file = json.load(f)
                        yield {'path':comments_path, 'content':issue_file, 'n-comments':len(comments_file)}
                except:
                    continue
                
def store_labeled_issues(batch, predictions):
    for i, issue in enumerate(batch):
        issue['content']['td-label-li2022-emse'] = str(predictions[i])
        with open(Path(issue['path']).joinpath('json'), 'w') as f:
            json.dump(issue['content'], f)


In [5]:
from IPython.display import clear_output
from minirig import load_csv_dataset
import pandas as pd
import json
import os

batch_size = 128
bots_dataset = pd.read_csv('../data/bots-dataset-labeling-management.csv',delimiter=',')

for i in bots_dataset.index:
    cnt = 1
    if bots_dataset['label_finished'][i] == 'yes':
        continue
    bot = bots_dataset['account'][i]
    batch = []
    for issue in load_issues_per_bot(bot, bots_dataset['issue_count'][i]):
        if cnt % 1500 == 0:
            model1_li2022_emse.clear_model_session()
        if 'body' in issue['content'].keys() and issue['content']['body'] != None and 'td-label-li2022-emse' not in issue['content'].keys():
            batch.append(issue)
        if len(batch) >= batch_size:
            predictions = model1_li2022_emse.label_sections_in_batch(comments=[x['content']['body'] for x in batch], batch_size=batch_size)
            store_labeled_issues(batch, predictions)
            batch = []         
        clear_output()
        logging.info(f'{bot}: {cnt}/{bots_dataset["issue_count"][i]} (len batch:{len(batch)})')
        cnt += 1
    if len(batch) > 0:
        predictions = model1_li2022_emse.label_sections_in_batch(comments=[x['content']['body'] for x in batch], batch_size=batch_size)
        store_labeled_issues(batch, predictions)
        batch = []
    bots_dataset['label_finished'][i] = 'yes'
    bots_dataset.to_csv('../data/bots-dataset-labeling-management.csv',)

Exception ignored in: <generator object load_issues_per_bot at 0x7fb7b804d1c0>
Traceback (most recent call last):
  File "/tmp/ipykernel_15536/1157556291.py", line 22, in <module>
RuntimeError: generator ignored GeneratorExit
ERROR:tornado.application:Exception in callback functools.partial(<bound method OutStream._flush of <ipykernel.iostream.OutStream object at 0x7fb82f1d6ad0>>)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/jupyter_client/session.py", line 95, in json_packer
    return json.dumps(
UnicodeEncodeError: 'utf-8' codec can't encode character '\ud83d' in position 20607: surrogates not allowed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/tornado/ioloop.py", line 738, in _run_callback
    ret = callback()
  File "/opt/conda/lib/python3.10/site-packages/ipykernel/iostream.py", line 540, in _flush
    self.session.send(
  File "/opt/c

TypeError: getWordVector(): incompatible function arguments. The following argument types are supported:
    1. (self: fasttext_pybind.fasttext, arg0: fasttext_pybind.Vector, arg1: str) -> None

Invoked with: <fasttext_pybind.fasttext object at 0x7fb82f0b41f0>, <fasttext_pybind.Vector object at 0x7fb79c75edf0>, '\ud83d'

In [11]:
print ("\ud83d\ude04".encode('utf-16','surrogatepass').decode('utf-16'))

😄


## Step 7 - Labeling sections with [Li et al.](http://doi.org/10.1109/TSE.2022.3224378)'s Model 


In [ ]:
from IPython.display import clear_output
from minirig import GHRequests, load_csv_dataset
import pandas as pd
import json
from model_li2022_tse import *

v = Model1_IssueTracker_Li2022_TSE('../model/satd-issue_mul.hdf5', '../model/fasttext_issue_300.bin')

issues_dataset = pd.read_csv('../data/dataset-backup.csv')

In [ ]:
issues_dataset['td-label-li-tse'] = '-'
len_dataset = len(issues_dataset)
for j, i in enumerate(issues_dataset.index):
    issues_dataset['td-label'][i] = v.classify_prob_comment(issues_dataset['text'][i])
    clear_output()
    log.info(f'Handling {i+1} out {len_dataset} lines')